# Test interfaccia per produrre tabella di vertici usando i file MC a disposizione
Scrivo qui tutti i comandi in modo da non dimenticarmi quale script ho usato.
La tabella di vertici è salvata in un file CSV (Comma Separated Values), con il primo rigo con gli indici
Sunto degli script usati
1. Salvataggio indici di charm e figli di charm dalla simulazione MC
2. Identificazione vertici primari da MC
3. Identificazione vertici con figlie di charm

In [1]:
#variables with file paths
macropath = "/home/antonio/Dottorato/Analisi/macros-ship"
workdir = "/home/antonio/Dottorato/Charmdata/CH1-R6/decay_search_MC"

#INPUT FILES
inputsimtree = workdir+"/ship.conical.Pythia8CharmOnly-TGeant4.root"
inputvertextree = workdir+"/vertextree_newformat.root"

#OUTPUT FILES
charmlistfile = workdir+"/charmlist.p"
outputcsv = workdir+"/MC_vertexlist_withallprimaries.csv"

In [2]:
!echo $charmlistfile

/home/antonio/Dottorato/Charmdata/CH1-R6/decay_search_MC/charmlist.p


## Salvataggio indici MC
Per evitare di dover tenere aperto il file MC con la simulazione originaria, salvo delle liste con gli indici delle tracce di charm e figlie di charm in un file Python-Pickle. Salvo le seguenti liste:

1. MCTrackID di adroni charmti
2. MCTrackID di figli di charm carichi
3. numero di figli di charm carichi

N.B. è fondamentale che gli altri script leggano queste liste **nello stesso ordine** in cui sono state scritte. Infatti Pickle non usa dei nomi per riconoscere gli oggetti salvati, ma li legge basandoli solo sull'ordine di scrittura e di lettura

In [3]:
run $macropath"/analisi_charmsim/writecharmdaughters.py" -s $inputsimtree -co $charmlistfile

Welcome to JupyROOT 6.18/00
Testing storage


## Identificazione vertici primari
Dato il formato della simulazione FairShip di ship-charm, le particelle prodotte al vertice primario hanno MotherID uguale a -1 (esclusi i due charm, che hanno MotherID 0). Pertanto posso riconoscere i primari nel MC richiedendo che la maggior parte delle tracce associate abbiamo MotherID -1. 

Se ci sono più vertici nello stesso evento con questa caratteristica (primari associati a vertici diversi), salvo solo il vertice con più tracce.

In [4]:
run $macropath"/analisi_charmsim/search_primary_vertices.py" -f $inputvertextree -o $outputcsv

Load FEDRA libs


Error in <TUnixSystem::Load>: version mismatch, /home/antonio/fedra/lib/libEdr.so = 61600, ROOT = 61800
Error in <TUnixSystem::Load>: version mismatch, /home/antonio/fedra/lib/libEdd.so = 61600, ROOT = 61800


## Identificazione vertici secondari
Ora utilizzo le liste salvate dallo script 1. Faccio un loop sulle tracce associate ai vertici: se MCTrackID corrisponde a una figlia di charm, salvo indici di traccia e vertice nel file csv in uscita.

In [5]:
run $macropath"/analisi_charmsim/search_secondary_vertices.py" -f $inputvertextree -c $charmlistfile -o $outputcsv 

## Controllo del file prodotto e ordinamento

Tutti gli script sono stati eseguiti. Ordino la lista per evento e topologia di vertice:

1. Primari
2. Vertici con figlie di charm

Posso controllare le prime e ultime righe per vedere se ci sono cose strane.

In [6]:
import pandas as pd
df = pd.read_csv(outputcsv)

In [7]:
df = df.sort_values(by=['MCEventID','topology'])

In [8]:
df.to_csv(outputcsv,index=False)

In [9]:
df

,ntracks,ivtx,itrk,MCEventID,MCTrackID,MCMotherID,predmolt,quantity,vx,vy,vz,topology
12986,18,23473,39,0,2220,1,4,1,48039,47472,-11087,2
12987,18,23473,46,0,2384,1,4,1,48039,47472,-11087,2
12988,18,23473,45,0,2380,1,4,1,48039,47472,-11087,2
12989,18,23473,38,0,2099,2,2,1,48039,47472,-11087,2
12990,18,23473,29,0,1315,2,2,1,48039,47472,-11087,2
12991,18,23473,44,0,2378,1,4,1,48039,47472,-11087,2
10048,11,26065,51,1,10,-1,0,0,19688,50190,-19958,1
10049,11,26065,58810,1,2179,-1,0,0,19688,50190,-19958,1
10050,11,26065,58,1,3,-1,0,0,19688,50190,-19958,1
10051,11,26065,41648,1,4,-1,0,0,19688,50190,-19958,1
